<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/loaddata.xlsx")
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [ ]:
#forming  the dataset 
x = []
y = []
for i in range(1, data.shape[0]):
    x.append(data['Load (kW)'].iloc[i-1]) #Since x is previous hour of y
    y.append(data['Load (kW)'].iloc[i])
data = pd.DataFrame({'Previous Hour Load in kw' : x, 'Present Hour Load in kw' : y})
data.head()

,Previous Hour Load in kw,Present Hour Load in kw
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


In [ ]:
#normalizing
maxX = np.max(data['Previous Hour Load in kw'])
minX = np.min(data['Previous Hour Load in kw'])
maxY = np.max(data['Present Hour Load in kw'])
minY = np.min(data['Present Hour Load in kw'])

data['Previous Hour Load in kw'] = (data['Previous Hour Load in kw'] - minX) / (maxX - minX)
data['Present Hour Load in kw'] = (data['Present Hour Load in kw']- minY) / (maxY - minY)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['Previous Hour Load in kw']
y = data['Present Hour Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
m = np.random.uniform(-5, 5)
c = np.random.uniform(-5, 5)
eta = 0.01
epochs = 200
vm = 0
vc = 0
v = 0.5

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - (m+v*vm) * trainX.iloc[i] - c-v*vc) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - (m+v*vm) * trainX.iloc[i] - c-v*vc)
        
        # Calculating updated values of vm and vc
        vm = (v * vm) - (eta * gradM)
        vc = (v * vc) - (eta * gradC)
        
        # Updating m and c values
        m += vm
        c += vc

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.7794609906177262
c = 0.11440999815807879


In [ ]:

train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 656.9897491650023
Mean square error 431635.53050789266
Mean absolute error 473.16774805643155
testing error
Root mean square error 737.2925907233075
Mean square error 543600.3643354867
Mean absolute error 518.0933736868398


In [ ]:
#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})

,Actual value,Predicted value
0,7272.11376,6809.732174
1,6082.74936,6561.624315
2,6436.59696,6635.983771
3,5349.80160,6275.121705
4,8121.65976,7462.199950
...,...,...
1959,5671.84968,5760.923636
1960,5442.86196,5969.664724
1961,4870.93824,5506.983664
1962,8138.65068,7780.050174


In [ ]:

#checking the difference between actal value and predicted value of testing data
pd.DataFrame({'Actual value' : testY, 'Predicted value' : test_pred})

,Actual value,Predicted value
0,6052.82040,5887.529115
1,5190.80400,5712.808694
2,5097.27600,5478.309102
3,6263.57016,6903.289006
4,4543.90200,4992.056581
...,...,...
214,6172.84800,6529.061678
215,5864.20560,5337.973854
216,4667.82660,4241.414882
217,7943.64480,7758.665755


In [ ]:

x = float(input('Enter the load of previous  hour : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted load at present hour :', prediction)

Enter the load of previous  hour : 5432
Predicted load at present hour : 5604.102757099458
